In [18]:
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, year, month, dayofmonth, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType
import pandas as pd 
import numpy as np

In [20]:
import sys
import os
project_root = os.path.abspath('..')
print(f"root:{project_root}")
if project_root not in sys.path:
    sys.path.append(project_root)

root:c:\Users\romul\Repositories\Airbnb_Data_Warehouse


In [21]:
fact_chuvas = pd.read_csv(f"{project_root}/data/raw/dados_pluviometricos_consolidados.csv")

C:\Users\romul\AppData\Local\Temp\ipykernel_20660\3938555641.py:1: DtypeWarning: Columns (3,4,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  fact_chuvas = pd.read_csv(f"{project_root}/data/raw/dados_pluviometricos_consolidados.csv")


MemoryError: Unable to allocate 708. MiB for an array with shape (7, 13259456) and data type float64

In [ ]:
listings = pd.read_csv(f"{project_root}/data/raw/listings.csv")

In [ ]:
listings[listings['host_is_superhost']=='t']

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
1,25026,https://www.airbnb.com/rooms/25026,20250624193519,2025-07-04,city scrape,Beautiful Modern Decorated Studio in Copacabana,"**Fully renovated in Dec 2022, new kitchen, n...",Copacabana is a lively neighborhood and the ap...,https://a0.muscache.com/pictures/7c08fa4f-1d7b...,102840,...,4.93,4.85,4.65,NaN,f,1,1,0,0,1.71
2,35764,https://www.airbnb.com/rooms/35764,20250624193519,2025-06-25,city scrape,COPACABANA SEA BREEZE - RIO - 25 X Superhost,Our newly renovated studio is located in the b...,Our guests will experience living with a local...,https://a0.muscache.com/pictures/23782972/1d3e...,153691,...,4.95,4.95,4.89,NaN,f,1,1,0,0,2.88
3,48305,https://www.airbnb.com/rooms/48305,20250624193519,2025-06-26,city scrape,Bright 6bed Penthouse Seconds from Beach,Enter Bossa Nova's history by staying in the v...,Enter Bossa Nova history by staying in the ver...,https://a0.muscache.com/pictures/miso/Hosting-...,70933,...,4.84,4.95,4.59,NaN,t,6,5,1,0,1.05
5,49179,https://www.airbnb.com/rooms/49179,20250624193519,2025-06-27,city scrape,Djalma Ocean View Studio with Amazing Views,Beautiful 12th Floor Ocean View Studio steps f...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,224192,...,4.93,4.91,4.75,NaN,t,11,11,0,0,1.19
7,51703,https://www.airbnb.com/rooms/51703,20250624193519,2025-06-27,city scrape,"Ocean view, block of the beach, Copacabana,Ipa...",Oh....i love to host and to meet people intere...,"It is located in the hotspot of Rio, wonderful...",https://a0.muscache.com/pictures/d0295650-2b1e...,238091,...,4.92,4.93,4.67,NaN,f,2,2,0,0,1.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42553,1449836422986444435,https://www.airbnb.com/rooms/1449836422986444435,20250624193519,2025-06-28,city scrape,Mini apartamento em Copacabana,This room is an independent space in a well lo...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,27581763,...,NaN,NaN,NaN,NaN,t,3,3,0,0,NaN
42560,1450014170108315634,https://www.airbnb.com/rooms/1450014170108315634,20250624193519,2025-06-25,city scrape,Private room in Arporador,You'll have a great time at this comfortable p...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,45156129,...,NaN,NaN,NaN,NaN,f,4,1,3,0,NaN
42565,1450082923922623983,https://www.airbnb.com/rooms/1450082923922623983,20250624193519,2025-06-26,city scrape,Cama em Dorm Misto (9) c/Varanda,"Single bed in room with nine beds, located on ...",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,37776540,...,NaN,NaN,NaN,NaN,t,8,0,4,4,NaN
42566,1450108079751966655,https://www.airbnb.com/rooms/1450108079751966655,20250624193519,2025-06-27,city scrape,Apart 1 Quarto Rua Sá Ferreira Posto 6 Copa/ M...,"Apartment posto 6 of Copacabana, cozy atmosphe...",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,4232062,...,NaN,NaN,NaN,NaN,t,8,8,0,0,NaN


In [ ]:
len(listings['id'])

42572

In [ ]:
#callendar = pd.read_csv(f"{project_root}/data/raw/calendar.csv")

In [ ]:
reviews = pd.read_csv(f"{project_root}/data/raw/reviews.csv")

In [ ]:
len(reviews.listing_id.unique())

33949

## Criando notas artificias para as reviews.

In [ ]:
media_n = 0.6
desvio_n = 0.2
nota_max  =5
nota_min =0
def distribuition_generator(elements =len(reviews), media_n = 0.6,desvio_n = 0.2,nota_max  =5 ,nota_min =0):
    distribuicao = np.random.normal(media_n,desvio_n,elements)
    distribuicao = np.round(distribuicao*nota_max,decimals=1)
    distribuicao = np.clip(distribuicao,nota_min,nota_max)
    return pd.Series(distribuicao.flatten())

In [ ]:
notas_artificiais = {
    'nota_comunicacao': [0.4,0.3],
    'nota_limpeza' : [0.6,0.2],
    'nota_custo_beneficio' : [0.6,0.2]
}
for dist in notas_artificiais:
    media = notas_artificiais[dist][0]
    desvio = notas_artificiais[dist][1]
    reviews[dist] = distribuition_generator(media_n= media,desvio_n= desvio)    

In [ ]:
reviews.reset_index(drop=True)

,listing_id,id,date,reviewer_id,reviewer_name,comments,nota_comunicacao,nota_limpeza,nota_custo_beneficio
0,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...,2.5,4.0,3.9
1,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...,2.5,2.8,3.7
2,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...,0.8,3.3,3.2
3,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r<b...,3.9,1.5,2.0
4,17878,147594,2010-12-01,219338,David,The apt was nice and in a great location only ...,0.5,2.1,2.4
...,...,...,...,...,...,...,...,...,...
1069272,1447261809655430116,1449312583557631186,2025-06-22,502578145,Lylyan,"Eu e meu namorados adoramos a estadia, a local...",2.4,3.0,3.0
1069273,1447342549635319128,1450044909045473576,2025-06-23,523455314,Alice,Ambiente super acolhedor. O André e a Andressa...,0.0,2.7,3.7
1069274,1448141046834354537,1449348675550278871,2025-06-22,261457678,Thiago,"Ap muito lindo, ficamos apaixonada tudo organi...",0.0,4.0,2.1
1069275,1448141046834354537,1452327802375005883,2025-06-26,703349982,Ramon,O ap é lindo. Amamos cada detalhe. Tudo estava...,2.3,3.4,2.0


## Retirando colunas redundantes csv das chuvas.